# AGYW_PREV DATIM

In [1]:
# load the modules
from layeringOfServices import DREAMS_MASTERSHEET
#multiples outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## AGYW_PREV

In [221]:
class AgywPrev:
    """A class with properties and methods given the results of the indicator AGYW_PREV DATIM""" 
    __who_am_I ="DATIM"
    __datim1_title="Number of individual AGYW that have fully completed the entire DREAMS primary package of services but have not received any services beyond the primary package."
    __datim2_title="Number of individual AGYW that have fully completed the entire DREAMS primary package of services AND at least one additional secondary service."
    __datim3_title="Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services."
    
    
    @classmethod
    def datim_titleI(cls):
        return cls.__datim1_title
    
    @classmethod
    def datim_titleII(cls):
        return cls.__datim2_title
    
    @classmethod
    def datim_titleIII(cls):
        return cls.__datim3_title
    
    
    def __init__(self,commune=None):
        self.__commune = commune
        self.__i_am = f"{AgywPrev.__who_am_I}"
        self.__data =  DREAMS_MASTERSHEET
        self. __total_mastersheet = self.__data.code.count()
        if self.__commune == None:
            self.__dreams_valid = self.__data[(self.__data.age_range !="not_valid_age") & (self.__data.age_range != "25-29") & (self.__data.timeOn_system == "required_Time_on")]
        else:
            self.__dreams_valid = self.__data[(self.__data.age_range !="not_valid_age") & (self.__data.age_range != "25-29") & (self.__data.timeOn_system == "required_Time_on")&(self.__data.actual_commune == f"{self.__commune}")]
        self.__total_dreams_valid = self.__dreams_valid.code.count()
        self.__dreams_valid["primary_only"] = self.__dreams_valid.apply(lambda df: self.__primFunc(df),axis=1)
        self.__dreams_valid["primary_and_OneSecondary_services"] = self.__dreams_valid.apply(lambda df: self.__primLeastOneSecFunc(df),axis=1)
        self.__dreams_valid["partial_primary_services"] = self.__dreams_valid.apply(lambda df: self.__primPartFunc(df),axis=1)
        self.__agyw_prevI = self.__dreams_valid[self.__dreams_valid.primary_only == "full_primary_only"]
        self.__agyw_prevII = self.__dreams_valid[self.__dreams_valid.primary_and_OneSecondary_services == "full_primary_leastOneSecondary"]
        self.__agyw_prevIII = self.__dreams_valid[self.__dreams_valid.partial_primary_services == "primary_part_services"]  
        self.__agyw_prevI_total = self.__agyw_prevI.code.count()
        self.__agyw_prevII_total = self.__agyw_prevII.code.count()
        self.__agyw_prevIII_total = self.__agyw_prevIII.code.count()
        self.__total_datim = self.__agyw_prevI_total + self.__agyw_prevII_total + self.__agyw_prevIII_total
        
        
    @property
    def who_am_i(self):
        return self.__i_am
    
   @property
    def mastersheet_dataframe(self):
        return self.__data

    @property
    def total_mastersheet(self):
        return self.__total_mastersheet

    @property
    def total_dreams_valid(self):
        return self.__total_dreams_valid
    
    def __primFunc(self,df):
        if (df.ps_10_14 == "curriculum-servis" and df.hts == "errata" and df.condoms == "errata" and df.post_care_treatment == "no" and df.socio_eco_app == "no" ):
            return "full_primary_only"
        elif (df.ps_15_19 == "condoms&curriculum" and df.hts == "errata"  and df.post_care_treatment == "no" and df.socio_eco_app == "no" ):
            return "full_primary_only"
        elif (df.ps_20_24 == "condoms&hts&curriculum" and df.post_care_treatment == "no" and df.socio_eco_app == "no" ):
            return "full_primary_only"
        else:
            return "invalid"
    
    def __primLeastOneSecFunc(self,df):
        if (df.ps_10_14 == "curriculum-servis") and (df.hts == "tested_on_given_date" or df.condoms == "tested_on_given_date" or df.post_care_treatment == "service_gyneco_vbg" or df.socio_eco_app == "service_muso_gardening" ):
            return "full_primary_leastOneSecondary"
        elif (df.ps_15_19 == "condoms&curriculum") and (df.hts == "tested_on_given_date" or df.post_care_treatment == "service_gyneco_vbg" or df.socio_eco_app == "service_muso_gardening" ):
            return "full_primary_leastOneSecondary"
        elif (df.ps_20_24 == "condoms&hts&curriculum") and (df.post_care_treatment == "service_gyneco_vbg" or df.socio_eco_app == "service_muso_gardening" ):
            return "full_primary_leastOneSecondary"
        else:
            return "invalid"
        
    def __primPartFunc(self,df):
        if (df.age_range == "15-19") and ((df.condoms == "tested_on_given_date" and df.dreams_curriculum == "curriculum_inc") or (df.condoms == "errata" and df.dreams_curriculum == "curriculum_completed")):
            return "primary_part_services"
        elif (df.age_range == "20-24") and ((df.dreams_curriculum == "curriculum_completed" and df.condoms=="errata" and df.hts=="errata")or(df.dreams_curriculum == "curriculum_completed" and df.condoms=="tested_on_given_date" and df.hts=="errata")or(df.dreams_curriculum == "curriculum_completed" and df.condoms=="errata" and df.hts=="tested_on_given_date")or(df.dreams_curriculum == "curriculum_inc" and df.condoms=="tested_on_given_date" and df.hts=="errata")or(df.dreams_curriculum == "curriculum_inc" and df.condoms=="errata" and df.hts=="tested_on_given_date")or(df.dreams_curriculum == "curriculum_inc" and df.condoms=="tested_on_given_date" and df.hts=="tested_on_given_date")):
            return "primary_part_services"
        else:
            return "invalid"
         
    @property
    def total_datimI(self):
        return self.__agyw_prevI_total
    
    @property
    def total_datimII(self):
        return self.__agyw_prevII_total
    
    @property
    def total_datimIII(self):
        return self.__agyw_prevIII_total
    
    @property
    def total_datim_general(self):
        return self.__total_datim


    def datim_agyw_prevI(self):
        agyw_prevI_pivot = self.__agyw_prevI.pivot_table(index=["age_range"],columns=["agyw_period_range"], values="code",aggfunc="count",fill_value=0,margins=True,margins_name="Total")
        agyw_prevI_pivot.rename_axis(index={"age_range":"Age"},columns={"agyw_period_range":"Time"},inplace=True)
        agyw_prevI_pivot_final = agyw_prevI_pivot[:-1]
        return agyw_prevI_pivot_final
    
    def datim_agyw_prevII(self):
        agyw_prevII_pivot = self.__agyw_prevII.pivot_table(index=["age_range"],columns=["agyw_period_range"], values="code",aggfunc="count",fill_value=0,margins=True,margins_name="Total")
        agyw_prevII_pivot.rename_axis(index={"age_range":"Age"},columns={"agyw_period_range":"Time"},inplace=True)
        agyw_prevII_pivot_final = agyw_prevII_pivot[:-1]
        return agyw_prevII_pivot_final
    
    def datim_agyw_prevIII(self):
        agyw_prevIII_pivot = self.__agyw_prevIII.pivot_table(index=["age_range"],columns=["agyw_period_range"], values="code",aggfunc="count",fill_value=0,margins=True,margins_name="Total")
        agyw_prevIII_pivot.rename_axis(index={"age_range":"Age"},columns={"agyw_period_range":"Time"},inplace=True)
        agyw_prevIII_pivot_final = agyw_prevIII_pivot[:-1]
        return agyw_prevIII_pivot_final

In [222]:
%%capture
datim = AgywPrev()

In [223]:
datim.total_mastersheet 
datim.total_dreams_valid
datim.who_am_i
datim.total_datimI
datim.total_datimII
datim.total_datimIII
datim.total_datim_general
AgywPrev.datim_titleI()
datim.datim_agyw_prevI()
AgywPrev.datim_titleII()
datim.datim_agyw_prevII()
AgywPrev.datim_titleIII()
datim.datim_agyw_prevIII()

23325

15187

'DATIM'

5171

920

...

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
10-14,414,3742,183,4339
15-19,70,605,42,717
20-24,0,112,3,115


'Number of individual AGYW that have fully completed the entire DREAMS primary package of services AND at least one additional secondary service.'

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
10-14,24,586,31,641
15-19,0,249,15,264
20-24,0,15,0,15


'Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services.'

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
15-19,369,3401,201,3971
20-24,226,2179,155,2560


## AGYW_Prev_Commune

In [224]:
class AgywPrevCommune(AgywPrev):
    """A class that extend AgywPrev with the purpose of the indicator AGYW_PREV DATIM by commune"""
    __who_am_I ="DATIM"
    def __init__(self,name):
        self.__name = name
        self.__i_am = f"{AgywPrevCommune.__who_am_I} {self.__name}"
        #self.__data =  DREAMS_MASTERSHEET
        super().__init__(self.__name)
        
    @property
    def who_am_i(self):
        return self.__i_am
        
    
        
        

        

In [227]:
DREAMS_MASTERSHEET.actual_commune.unique()

array(['Verrettes', 'Delmas', 'Port-au-Prince', 'Carrefour', 'Limonade',
       'Plaine-du-Nord', 'Cap-Haïtien', 'Kenscoff', 'Dessalines',
       'Pétionville', 'Gressier', 'Saint-Marc', 'Tabarre', 'La Chapelle',
       'Milot', 'Liancourt', 'Quartier-Morin',
       "Petite Rivière de l'Artibonite", 'Grande Saline', 'Montrouis',
       'Desdunes'], dtype=object)

## dynamic

In [239]:
%%capture
for columns in DREAMS_MASTERSHEET.actual_commune.unique():
    globals()[f"datim_{columns.replace('-','_')}"] = AgywPrevCommune(f"{columns}")

In [240]:
datim_Port_au_Prince.total_mastersheet 
datim_Port_au_Prince.total_dreams_valid
datim_Port_au_Prince.who_am_i
datim_Port_au_Prince.total_datimI
datim_Port_au_Prince.total_datimII
datim_Port_au_Prince.total_datimIII
datim_Port_au_Prince.total_datim_general
AgywPrevCommune.datim_titleI()
datim_Port_au_Prince.datim_agyw_prevI()
AgywPrevCommune.datim_titleII()
datim_Port_au_Prince.datim_agyw_prevII()
AgywPrevCommune.datim_titleIII()
datim_Port_au_Prince.datim_agyw_prevIII()

23325

545

'DATIM Port-au-Prince'

175

14

...

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
10-14,4,150,2,156
15-19,2,12,0,14
20-24,0,5,0,5


'Number of individual AGYW that have fully completed the entire DREAMS primary package of services AND at least one additional secondary service.'

Time,07-12 months,Total
Age,,
10-14,11,11
15-19,3,3


'Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services.'

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
15-19,14,183,7,204
20-24,4,92,6,102


In [242]:
datim_Pétionville.total_mastersheet 
datim_Pétionville.total_dreams_valid
datim_Pétionville.who_am_i
datim_Pétionville.total_datimI
datim_Pétionville.total_datimII
datim_Pétionville.total_datimIII
datim_Pétionville.total_datim_general
AgywPrevCommune.datim_titleI()
datim_Pétionville.datim_agyw_prevI()
AgywPrevCommune.datim_titleII()
datim_Pétionville.datim_agyw_prevII()
AgywPrevCommune.datim_titleIII()
datim_Pétionville.datim_agyw_prevIII()

23325

369

'DATIM Pétionville'

85

11

...

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
10-14,18,52,9,79
15-19,0,3,0,3
20-24,0,3,0,3


'Number of individual AGYW that have fully completed the entire DREAMS primary package of services AND at least one additional secondary service.'

Time,07-12 months,13-24 months,Total
Age,,,
10-14,4,2,6
15-19,4,0,4
20-24,1,0,1


'Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services.'

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
15-19,18,91,11,120
20-24,11,45,7,63


In [244]:
datim_Delmas.total_mastersheet 
datim_Delmas.total_dreams_valid
datim_Delmas.who_am_i
datim_Delmas.total_datimI
datim_Delmas.total_datimII
datim_Delmas.total_datimIII
datim_Delmas.total_datim_general
AgywPrevCommune.datim_titleI()
datim_Delmas.datim_agyw_prevI()
AgywPrevCommune.datim_titleII()
datim_Delmas.datim_agyw_prevII()
AgywPrevCommune.datim_titleIII()
datim_Delmas.datim_agyw_prevIII()

23325

481

'DATIM Delmas'

149

23

...

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
10-14,42,65,17,124
15-19,0,18,5,23
20-24,0,2,0,2


'Number of individual AGYW that have fully completed the entire DREAMS primary package of services AND at least one additional secondary service.'

Time,07-12 months,13-24 months,Total
Age,,,
10-14,10,5,15
15-19,4,4,8


'Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services.'

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
15-19,34,103,28,165
20-24,17,66,25,108


In [245]:
datim_Carrefour.total_mastersheet 
datim_Carrefour.total_dreams_valid
datim_Carrefour.who_am_i
datim_Carrefour.total_datimI
datim_Carrefour.total_datimII
datim_Carrefour.total_datimIII
datim_Carrefour.total_datim_general
AgywPrevCommune.datim_titleI()
datim_Carrefour.datim_agyw_prevI()
AgywPrevCommune.datim_titleII()
datim_Carrefour.datim_agyw_prevII()
AgywPrevCommune.datim_titleIII()
datim_Carrefour.datim_agyw_prevIII()

23325

1255

'DATIM Carrefour'

521

32

...

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
10-14,35,402,24,461
15-19,3,48,6,57
20-24,0,3,0,3


'Number of individual AGYW that have fully completed the entire DREAMS primary package of services AND at least one additional secondary service.'

Time,07-12 months,13-24 months,Total
Age,,,
10-14,26,2,28
15-19,4,0,4


'Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services.'

Time,0-6 months,07-12 months,13-24 months,Total
Age,,,,
15-19,32,265,49,346
20-24,21,151,24,196
